In [ ]:
import requests
import pandas as pd

# 게임 제목을 Steam Store API에서 가져오는 함수
def get_game_title(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        # 데이터 구조: { "240720": { "success": True, "data": {"name": "...", ... } } }
        app_data = data.get(str(app_id))
        if app_data and app_data.get('success') and 'data' in app_data:
            return app_data['data'].get('name', "Unknown Title")
    return "Unknown Title"

# 게임의 리뷰와 추천유무, 댓글공감수, 작성 시간(timestamp_created)를 추출
def get_steam_reviews(app_id, num_pages=50):
    reviews = []
    cursor = "*"
    
    for page in range(num_pages):
        url = (
            f"https://store.steampowered.com/appreviews/{app_id}"
            f"?json=1&num_per_page=100&language=english&purchase_type=all"
            f"&cursor={cursor}&filter=all"
        )
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            
            # 응답 데이터 구조 확인 (디버그 용)
            if "reviews" not in data:
                print(f"응답에 'reviews' 키가 없습니다. page {page}의 데이터 키:", list(data.keys()))
                break
            
            for review in data['reviews']:
                reviews.append({
                    "review": review.get("review", ""),
                    "voted_up": review.get("voted_up", False),
                    "votes_up": review.get("votes_up", 0),
                    "timestamp_created": review.get("timestamp_created", 0)  # Unix timestamp
                })
            # 커서(cursor) 갱신
            cursor = data.get('cursor', "")
            if not cursor:
                print("더 이상 가져올 리뷰가 없습니다. (cursor 없음)")
                break
        else:
            print(f"page {page} 에러, 상태 코드: {response.status_code}")
            break

    return pd.DataFrame(reviews)

# appID: Getting Over It with Bennett Foddy (가정: 240720)
app_id = 240720

# 1. 게임 제목 출력
game_title = get_game_title(app_id)
print("게임명:", game_title)

# 2. 리뷰 데이터 DataFrame 추출 (전체 기간의 리뷰를 수집하기 위해 num_pages를 50으로 설정)
reviews_df = get_steam_reviews(app_id, num_pages=50)

if reviews_df.empty:
    print("리뷰 데이터가 없습니다.")
else:
    # timestamp_created를 datetime 형식으로 변환
    reviews_df['datetime'] = pd.to_datetime(reviews_df['timestamp_created'], unit='s')
    
    # datetime을 인덱스로 설정하여 resample 기능 사용
    reviews_df.set_index('datetime', inplace=True)

    # 전체 기간의 월별 리뷰 수 집계 ('M'은 월별 리샘플링)
    monthly_review_counts = reviews_df.resample('M').size()

    # 전체 기간의 주별 리뷰 수 집계 ('W'는 주별 리샘플링)
    weekly_review_counts = reviews_df.resample('W').size()

    print("\n=== 월별 리뷰 수 ===")
    print(monthly_review_counts)

    print("\n=== 주별 리뷰 수 ===")
    print(weekly_review_counts)


게임명: Getting Over It with Bennett Foddy
응답에 'reviews' 키가 없습니다. page 1의 데이터 키: ['success', 'error']

=== 월별 리뷰 수 ===
datetime
2025-03-31    70
2025-04-30    30
Freq: ME, dtype: int64

=== 주별 리뷰 수 ===
datetime
2025-03-16    23
2025-03-23    25
2025-03-30    21
2025-04-06    25
2025-04-13     6
Freq: W-SUN, dtype: int64


C:\Users\user\AppData\Local\Temp\ipykernel_11160\3254106518.py:74: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_review_counts = reviews_df.resample('M').size()


In [23]:
import requests
import pandas as pd

def get_steam_reviews_from_cutoff(app_id, cutoff_date, max_pages=50):
    """
    지정한 cutoff_date(예: '2017-12-01') 이후의 리뷰만을 수집합니다.
    최신 리뷰부터 반환하므로, 기준일 이전의 리뷰가 나타나면 수집을 중단합니다.

    Parameters:
        app_id (int): Steam 앱 ID (예: 240720)
        cutoff_date (str): 시작 날짜 (예: '2017-12-01')
        max_pages (int): 시도할 최대 페이지 수 (기본값: 50)

    Returns:
        pd.DataFrame: cutoff_date 이후의 리뷰 데이터를 담은 DataFrame
    """
    reviews = []
    cursor = "*"
    # 기준 날짜를 Unix timestamp로 변환
    cutoff_ts = pd.Timestamp(cutoff_date).timestamp()
    
    for page in range(max_pages):
        url = (
            f"https://store.steampowered.com/appreviews/{app_id}"
            f"?json=1&num_per_page=100&language=english&purchase_type=all"
            f"&cursor={cursor}&filter=all"
        )
        response = requests.get(url)
        if response.status_code != 200:
            print(f"페이지 {page}에서 오류 발생: {response.status_code}")
            break

        data = response.json()
        reviews_in_page = data.get('reviews', [])
        
        # 리뷰가 없으면 중단
        if not reviews_in_page:
            break
        
        # 각 리뷰의 timestamp_created가 cutoff_ts보다 크거나 같아야 포함합니다.
        for review in reviews_in_page:
            timestamp = review.get("timestamp_created", 0)
            if timestamp >= cutoff_ts:
                reviews.append({
                    "timestamp_created": timestamp,
                    "review": review.get("review", ""),
                    "voted_up": review.get("voted_up", False),
                    "votes_up": review.get("votes_up", 0),
                })
            else:
                # 기준일보다 오래된 리뷰가 나타나면 이후 리뷰는 모두 오래된 것으로 보고 종료합니다.
                print("기준일보다 오래된 리뷰를 발견하여 수집을 종료합니다.")
                return pd.DataFrame(reviews)
        
        # 다음 페이지를 위한 커서 갱신
        cursor = data.get('cursor')
        if not cursor:
            break

    return pd.DataFrame(reviews)

# 앱 ID와 cutoff_date를 설정합니다.
app_id = 240720
cutoff_date = '2017-12-01'

# cutoff_date 이후의 리뷰를 수집합니다.
reviews_df = get_steam_reviews_from_cutoff(app_id, cutoff_date, max_pages=50)
print(f"{cutoff_date} 이후 수집된 총 리뷰 개수: {len(reviews_df)}")

# Unix timestamp를 datetime으로 변환
reviews_df['datetime'] = pd.to_datetime(reviews_df['timestamp_created'], unit='s')

# 2018년 1월의 리뷰만 필터링
jan_2018_reviews = reviews_df[
    (reviews_df['datetime'].dt.year == 2018) & (reviews_df['datetime'].dt.month == 1)
]

print("2018년 1월의 리뷰 개수:", len(jan_2018_reviews))


2017-12-01 이후 수집된 총 리뷰 개수: 100
2018년 1월의 리뷰 개수: 0


In [38]:
import requests
import pandas as pd

def get_steam_reviews_from_cutoff(app_id, cutoff_date, max_pages=50):
    """
    Steam API를 통해 지정한 cutoff_date 이후의 리뷰를 수집합니다.
    최신 리뷰부터 반환하며, cutoff_date보다 오래된 리뷰를 만나면 수집을 중단합니다.

    Parameters:
        app_id (int): Steam 앱 ID (예: 240720)
        cutoff_date (str): 리뷰 수집 기준 날짜 (예: '2017-12-01')
        max_pages (int): 시도할 최대 페이지 수 (기본값: 50)

    Returns:
        pd.DataFrame: 수집된 리뷰 데이터를 담은 DataFrame (timestamp_created, review, voted_up, votes_up 컬럼 포함)
    """
    reviews = []
    cursor = "*"
    # 기준 날짜를 Unix timestamp로 변환
    cutoff_ts = pd.Timestamp(cutoff_date).timestamp()
    
    for page in range(max_pages):
        url = (
            f"https://store.steampowered.com/appreviews/{app_id}"
            f"?json=1&num_per_page=100&language=english&purchase_type=all"
            f"&cursor={cursor}&filter=all"
        )
        response = requests.get(url)
        if response.status_code != 200:
            print(f"페이지 {page}에서 오류 발생: {response.status_code}")
            break

        data = response.json()
        reviews_in_page = data.get('reviews', [])
        
        if not reviews_in_page:  # 더 이상 리뷰가 없으면 중단
            break
        
        for review in reviews_in_page:
            timestamp = review.get("timestamp_created", 0)
            # 기준 날짜 이후의 리뷰만 추가
            if timestamp >= cutoff_ts:
                reviews.append({
                    "timestamp_created": timestamp,
                    "review": review.get("review", ""),
                    "voted_up": review.get("voted_up", False),
                    "votes_up": review.get("votes_up", 0),
                })
            else:
                # 기준 날짜보다 오래된 리뷰를 만나면 이후 리뷰는 모두 해당되지 않으므로 종료
                print("기준일보다 오래된 리뷰를 발견하여 수집을 종료합니다.")
                return pd.DataFrame(reviews)
        
        cursor = data.get('cursor')
        if not cursor:
            break

    return pd.DataFrame(reviews)


# ---------------------- 설정 ----------------------
app_id = 240720                       # 게임 'Getting Over It with Bennett Foddy'
cutoff_date = '2017-12-01'            # 리뷰 수집 기준 시작 날짜
target_year = 2022                    # 확인하려는 연도
target_month = 12                      # 확인하려는 월
max_pages = 5000                        # 최대 페이지 수 (필요에 따라 수정 가능)
# --------------------------------------------------

# 리뷰 데이터 수집 (cutoff_date 이후의 리뷰)
reviews_df = get_steam_reviews_from_cutoff(app_id, cutoff_date, max_pages=max_pages)

# Unix timestamp를 datetime 형식으로 변환
reviews_df['datetime'] = pd.to_datetime(reviews_df['timestamp_created'], unit='s')

# 지정한 연도와 월에 해당하는 리뷰만 필터링
filtered_reviews = reviews_df[
    (reviews_df['datetime'].dt.year == target_year) & (reviews_df['datetime'].dt.month == target_month)
]

# 결과 출력
print(f"{target_year}년 {target_month}월의 리뷰 개수: {len(filtered_reviews)}")


2022년 12월의 리뷰 개수: 0


In [1]:
import requests
import time
from datetime import datetime, timezone

# 여기에 Steam 게임의 AppID를 입력하세요.
app_id = "240720"  # 예: "570" (Dota 2의 경우)

# 리뷰 API 엔드포인트 설정
url = f"https://store.steampowered.com/appreviews/{app_id}?json=1"

# 분석할 기간: 2017년 1월 1일부터 현재까지 (UTC 기준)
start_date = datetime(2017, 1, 1, tzinfo=timezone.utc)
current_date = datetime.now(timezone.utc)

# (연도, 월) 별 리뷰 개수를 저장할 딕셔너리 초기화
monthly_counts = {}

# 페이징용 cursor 초기값 (첫 페이지 요청 시 "*")
cursor = "*"
num_per_page = 100  # 한 번에 가져올 리뷰 개수 (최대 100)

while True:
    params = {
        "cursor": cursor,
        "num_per_page": num_per_page,
        "language": "all"  # 모든 언어의 리뷰를 가져옵니다.
        # 필요시 "filter": "recent" 등의 옵션 추가 가능
    }
    
    response = requests.get(url, params=params)
    if response.status_code != 200:
        print("API 요청 실패:", response.status_code)
        break

    data = response.json()
    reviews = data.get("reviews", [])
    
    # 더 이상 리뷰가 없으면 종료
    if not reviews:
        break

    # 현재 페이지의 리뷰를 순회하며 처리
    reached_old_reviews = False  # 2017년 이전 리뷰를 만났는지 확인하기 위한 변수
    for review in reviews:
        ts = review.get("timestamp_created")
        review_time = datetime.fromtimestamp(ts, tz=timezone.utc)
        
        # 만약 2017년 1월 1일 이전의 리뷰라면 더 이상 진행하지 않습니다.
        if review_time < start_date:
            reached_old_reviews = True
            break
        
        # (연도, 월) 키 생성 후 리뷰 개수를 누적
        key = (review_time.year, review_time.month)
        monthly_counts[key] = monthly_counts.get(key, 0) + 1

    # 2017년 이전의 리뷰가 나타나면 루프 종료
    if reached_old_reviews:
        break

    # 다음 페이지를 위한 cursor 업데이트
    cursor = data.get("cursor")
    if not cursor:
        break

    # API에 부담을 주지 않기 위해 잠시 대기 (필요에 따라 조정)
    time.sleep(0.5)

# 2017년 1월부터 현재까지 순차적으로 순회하며 결과 출력 (해당 월에 리뷰가 없으면 0 출력)
year = start_date.year
month = start_date.month
while (year < current_date.year) or (year == current_date.year and month <= current_date.month):
    count = monthly_counts.get((year, month), 0)
    print(f"{year}년 {month}월 - {count}개")
    
    # 다음 월로 이동
    month += 1
    if month > 12:
        month = 1
        year += 1

ConnectionError: HTTPSConnectionPool(host='store.steampowered.com', port=443): Max retries exceeded with url: /appreviews/240720?json=1&cursor=AoMFQFueegAAAAAFP9555aAAAAB1zI3YBQ%3D%3D&num_per_page=100&language=all (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002BF0A415C50>: Failed to resolve 'store.steampowered.com' ([Errno 11001] getaddrinfo failed)"))